In [1]:
from csv import reader
import pandas as pd
import numpy as np

import random
import sys

import os
import shutil

# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

import time

import multiprocessing
from functools import partial

# genetic algorithm search of the one max optimization problem
from numpy.random import randint
# from numpy.random import rand


import Scripts.LevenClustering as levenClustering
import Scripts.SeqCleaning as seqCleaning
import Scripts.GenFunction as genFunction
import Scripts.GAFunctions as GAfunctions
import Scripts.StatFitness as statFitness

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

output = "Puzzles_Output"
# file = ['00 Result/SelectionF15/puzzle_1.csv', '00 Result/SelectionS16/puzzle_1.csv']
file = ['Puzzles_CSV/Puzzle_1.csv']

In [3]:
import numpy as np
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in range(len(data)):
        z_score = (data[i]-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [4]:
col_names = ['File Path', 'sequence', 'steps', 'time_spent']
df = pd.DataFrame(columns = col_names)

file_path = []
sequence = []
steps = []
time_spent = []

pop_cache = {}

if __name__ == "__main__":

    for f in file:

        with open(f, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)

            if header != None:
                for row in csv_reader:
                    file_path.append(row[0])
                    sequence.append(row[6])
                    steps.append(row[3])
                    time_spent.append(row[5])

    df['File Path'] = file_path
    df['sequence'] = sequence
    df['steps'] = steps
    df['time_spent'] = time_spent

    print("Total Rows for original: ", len(df.index))
    

    df['Extra_Steps'] = (df['steps'].astype(int))-16
    sample_outliers = detect_outliers_zscore(df['Extra_Steps'])
    update_df = df.copy()
    update_df = df.drop(sample_outliers)
    update_df = update_df.reset_index(drop=True)
    
    
    ###new_code
    num_R = genFunction.count_label(update_df, 'R')
    update_df['num_R'] = num_R
    num_T = genFunction.count_label(update_df, 'T')
    update_df['num_T'] = num_T
    
    #################

    
    print("Total Rows for after elimination outliers: ", len(update_df.index))
        

Total Rows for original:  294
Total Rows for after elimination outliers:  286


In [5]:
job_id = "standard"
n_bits = 26 # bits
label_mask = 15 # split the R and T

n_clusters = 2

# Keep All
cleaning_encode = genFunction.convert_to_binary(2**26-1, n_bits)

# Clear All
# cleaning_encode = genFunction.convert_to_binary(0, n_bits)

# Encode according to GA
# cleaning_encode = [1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]



df_1 = update_df

pop_cache = {}

result = genFunction.calc_score(df_1, pop_cache, label_mask, n_clusters, cleaning_encode)

print(result)
    
df_final = genFunction.dataset_cleaning(df_1, label_mask, cleaning_encode)
df_final, _ = GAfunctions.fitness(df_final, n_clusters, 1)

df_final.to_csv('{}/{}_best_final_output.csv'.format(output, job_id), index=False) 


(67108863, 0.39722664900321825, 749.1751312505737, 0.5572710210671592, 0.9064787537039531, [133, 115])


In [6]:
# file = '~/Desktop/GECCO/Visualization/GA Results/3 Clusters - AIC.csv'

In [7]:
df = df_final

In [8]:
# df.plot(kind='line',x='Generation',y='AVG_fitness',color='red')
# plt.title("AIC as GA Fitness (3 Clusters)")
# plt.xlabel("Average AIC in each generation")
# plt.ylabel("AIC")
# plt.savefig('AIC_3_clusters.png')
# plt.show()

KeyError: 'Generation'